In [1]:
import pandas as pd                                                         # importando a biblioteca pandas (utilizada para manipulação e análise de dados)
import numpy as np                                                          # importando a biblioteca numpy (usada para operações matemáticas e manipulação de arrays)

# Algoritmo Random Forest

## 1- Construção de Árvores de Decisão:

A Random Forest constrói múltiplas árvores de decisão durante o tempo de treinamento. Uma Árvore de Decisão é um modelo preditivo que divide repetidamente os dados em subconjuntos baseados em características (features) específicas, visando reduzir a impureza dos dados em cada divisão.

## 2- Agregação por Bootstrap (Bagging):

A técnica de agregação por bootstrap, ou simplesmente "bagging", é utilizada para treinar cada árvore. Isso envolve:

- Amostragem com reposição: Para cada árvore, uma amostra aleatória do conjunto de treinamento original é extraída com reposição. Isso significa que alguns dados podem ser selecionados várias vezes, enquanto outros podem não ser selecionados.
- Treinamento das Árvores: Cada árvore é treinada de forma independente usando essas amostras aleatórias.

## 3- Diversidade entre Árvores:

Para aumentar a diversidade entre as árvores e evitar que todas sigam os mesmos padrões de divisão, a Random Forest introduz aleatoriedade adicional:

- Subconjuntos de Características: Ao invés de considerar todas as características disponíveis para cada divisão, apenas um subconjunto aleatório de características é considerado. Esse subconjunto é selecionado aleatoriamente para cada divisão em cada árvore.

## 4- Combinação de Previsões:

Após o treinamento, as previsões de todas as árvores são combinadas para gerar a previsão final:
- Classificação: A classe final é determinada pela moda das previsões de todas as árvores (a classe mais votada).
- Regressão: A previsão final é a média das previsões de todas as árvores.

In [2]:
from random import seed, randrange  # seed: define a semente para o gerador de números aleatórios, garantindo que ele produza a mesma sequência de números toda vez que o código for executado;
                                    # randrange: gera um número inteiro aleatório dentro de um intervalo especificado;
from csv import reader              # reader: função usada para ler dados de um arquivo CSV;
from math import sqrt               # sqrt: função usada para calcular a raiz quadrada de um número.

In [3]:
# Carregando um arquivo CSV:

def load_csv(filename):                          # Define uma função chamada load_csv para carregar o arquivo CSV a ser lido.
    dataset = list()                             # Inicializa uma lista vazia chamada dataset, que será usada para armazenar as linhas do CSV.
    with open(filename, 'r') as file:            # Abre o arquivo CSV para leitura ('r'). *O with statement garante que o arquivo será fechado automaticamente após a leitura.
        csv_reader = reader(file)                # Cria um objeto 'csv_reader' usando a função 'reader' da biblioteca 'csv'. Este objeto permitirá iterar (percorrer ou passar por cada item de uma sequência de dados) sobre as linhas do arquivo CSV.
        for row in csv_reader:                   # Inicia um loop 'for' que itera sobre cada linha (row) no csv_reader.
            if not row: continue                 # Verifica se a linha (row) está vazia. Se estiver vazia, usa 'continue' para pular para a próxima iteração do loop, evitando adicionar linhas vazias ao dataset.
            dataset.append(row)                  # Adiciona a linha (row) não vazia à lista dataset.
    return dataset                               # Após ler todas as linhas do CSV e adicioná-las à lista dataset, a função retorna o dataset.

In [23]:
dataset = pd.read_csv("/content/car_accidentOFICIAL.csv")

In [5]:
# Convertendo os valores de uma coluna de strings para floats:

def str_column_to_flt(dataset, column):                          # Define a função 'chamada str_column_to_flt', que converte os valores de uma coluna específica em um dataset de strings para floats.
    for row in dataset:                                          # Inicia um loop 'for' que itera sobre cada linha (row) no dataset.
        row[column] = float( row[column].strip() )               # Para cada linha, acessa o valor na coluna especificada, remove quaisquer espaços em branco usando strip(), converte o valor para float, e atualiza o valor na linha.

In [6]:
# Convertendo os valores de uma coluna de strings para inteiros:

def str_column_to_int(dataset, column):
    class_values = [ row[column] for row in dataset ]
    unique = set( class_values )
    lookup = dict()
    for i, value in enumerate( unique ): lookup[value] = i
    for row in dataset: row[column] = lookup[row[column]]
    return lookup

In [7]:
# Dividindo um dataset em k folds para validação cruzada (usada para avaliar a capacidade de generalização de um modelo):

from random import randrange

def cross_validation_split(dataset, n_folds):            # Define uma função chamada cross_validation_split (dataset que será dividido em folds, número de folds desejado para a divisão do dataset).
                                                           # Inicialização das variáveis:
    dataset_split = list()                               # dataset_split: uma lista que armazenará os folds criados.
    dataset_copy = list(dataset)                         # dataset_copy: uma cópia do dataset original, feito para garantir que não haja alterações no dataset original durante a divisão.
    fold_size = len(dataset) / n_folds                   # fold_size: tamanho de cada fold (é calculado dividindo o número total de exemplos pelo número de folds desejado).

    for i in range(n_folds):                             # Inicia um loop que será executado n vezes, criando um fold em cada iteração.
        fold = list()                                    # Inicializa uma lista vazia para armazenar os exemplos do fold atual.
        while len(fold) < fold_size:                     # Enquanto o tamanho do fold atual for menor que o tamanho esperado (fold_size), um loop é executado.
            index = randrange(len(dataset_copy))         # Um índice aleatório é escolhido dentro do intervalo válido para o dataset atual (usando randrange(len(dataset_copy))).
            fold.append(dataset_copy.pop(index))         # O exemplo correspondente ao índice escolhido é removido de dataset_copy (usando pop(index)) e adicionado ao fold atual.

        dataset_split.append(fold)                       # Após preencher o fold atual com exemplos suficientes, ele é adicionado à lista de folds (dataset_split).
    return dataset_split                                 # Após criar todos os folds, a função retorna a lista de folds.


In [8]:
# Calculando a precisão do modelo de classificação:

def accuracy_metric(actual, predicted):                 # Define uma função chamada accuracy_metric (lista de valores reais, lista de valores previstos pelo modelo).
    correct = 0                                         # Inicializa um contador para o número de previsões corretas.
    for i in range(len(actual)):                        # Inicia um loop que itera sobre cada par de valores real e previsto.
        if actual[i] == predicted[i]: correct += 1      # Verifica se a previsão (predicted[i]) coincide com o valor real (actual[i]), e, se sim, incrementa o contador 'correct'.
    return 100. * correct / float( len(actual) )        # Retorna a precisão calculada como a porcentagem de previsões corretas em relação ao total de previsões.
                                                        # (multiplica por 100 para obter o valor em percentual)

In [9]:
# Avaliando o algoritmo de aprendizado de máquina usando divisão de validação cruzada:

def evaluate_algorithm(dataset, algorithm, n_folds, *args):      # Define uma função chamada evaluate_algorithm (dataset, algoritmo de aprendizado de máquina a ser avaliado, número de folds para a validação cruzada, argumentos adicionais que podem ser passados para o algoritmo).
    folds = cross_validation_split(dataset, n_folds)             # Usa a função cross_validation_split para dividir o dataset em k (número de subconjuntos) folds.
    scores = list()                                              # Inicializa uma lista vazia para armazenar as pontuações de precisão de cada fold.
    for fold in folds:                                           # Inicia um loop que itera sobre cada fold criado pela função cross_validation_split.

        train_set = list(folds)                                  # Cria o conjunto de treinamento combinando todos os folds, exceto o fold atual.
        train_set.remove(fold)
        train_set = sum(train_set, [])

        test_set = list()                                        # Cria o conjunto de teste para o fold atual, removendo o valor de destino para evitar vazamento de informações durante o teste.
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None

        predicted = algorithm(train_set, test_set, *args)        # Executa o algoritmo de aprendizado de máquina no conjunto de treinamento e teste, passando quaisquer argumentos adicionais especificados.

        actual = [row[-1] for row in fold]                       # Obtém os valores reais do conjunto de teste e calcula a precisão das previsões feitas pelo algoritmo.
        accuracy = accuracy_metric(actual, predicted)

        scores.append(accuracy)                                  # Adiciona a precisão à lista de pontuações.
    return scores                                                # Retorna a lista de pontuações de precisão para todos os folds.

In [10]:
# Dividindo um dataset em dois grupos com base no valor de um atributo específico:

def test_split(index, value, dataset):                          # Define uma função chamada test_split (índice do atributo usado para a divisão, valor do atributo que determina a divisão, dataset a ser dividido).
    left, right = list(), list()                                # Inicializa duas listas vazias para os grupos resultantes da divisão: um grupo à esquerda e outro à direita.
    for row in dataset:                                         # Inicia um loop que itera sobre cada linha do dataset.
        if row[index] < value: left.append(row)                 # Verifica se o valor do atributo na linha é menor que o valor especificado (value), se for, a linha é adicionada ao grupo à esquerda (left).
        else: right.append(row)                                 # Caso contrário, a linha é adicionada ao grupo à direita (right).
    return left, right                                          # Retorna os grupos resultantes à esquerda e à direita

In [11]:
# Calculando o índice Gini (métrica de impureza usada em algoritmos de árvore de decisão para avaliar a qualidade de uma divisão em um dataset) para uma divisão de dataset:

def gini_index(groups, class_values):                                                    # Define uma função chamada gini_index (lista de grupos após a divisão do dataset, lista dos valores de classe possíveis).
    gini = 0.0                                                                           # Inicializa o índice Gini como 0.
    for class_value in class_values:                                                     # Inicia um loop que itera sobre cada valor de classe possível.
        for group in groups:                                                             # Inicia um loop que itera sobre cada grupo na divisão do dataset.
            size = len(group)                                                            # Calcula o tamanho do grupo atual.
            if size == 0: continue                                                       # Verifica se o grupo está vazio. Se estiver, passa para o próximo grupo.
            proportion = [row[-1] for row in group].count(class_value) / float(size)     # Calcula a proporção de instâncias no grupo atual que têm o valor de classe atual.
            gini += (proportion * (1.0 - proportion))                                    # Adiciona a contribuição do grupo para o índice Gini. Quanto maior a impureza do grupo, maior será a contribuição para o índice Gini.
    return gini                                                                          # Retorna o índice Gini final.

In [12]:
# Selecionar o melhor ponto de divisão para o dataset: (objetivo é encontrar a melhor maneira de dividir o dataset com base em determinadas características (atributos) para maximizar a pureza dos grupos resultantes).

def get_split(dataset, n_features):                                                      # Define uma função chamada get_split (dataset para o qual o ponto de divisão será selecionado, número de atributos a serem considerados durante a seleção do ponto de divisão).
    class_values = list(set(row[-1] for row in dataset))                                 # Obtém os valores de classe únicos no dataset.
                                                                                            # Inicialização das variáveis para armazenar o melhor ponto de divisão encontrado até agora:
    b_index, b_value, b_score, b_groups = 999, 999, 999, None                            # b_index: Índice do atributo usado para a divisão. / b_value: Valor do atributo usado para a divisão. / b_score: Índice Gini do melhor ponto de divisão encontrado até agora. / b_groups: Grupos resultantes da melhor divisão encontrada até agora.

    features = list()                                                                    # Seleciona aleatoriamente 'n' índices de atributos para consideração durante a seleção do ponto de divisão.
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features: features.append(index)

    for index in features:                                                               # Inicia um loop que itera sobre cada índice de atributo selecionado.
        for row in dataset:                                                              # Inicia um loop que itera sobre cada linha no dataset.
            groups = test_split(index, row[index], dataset)                              # Divide o dataset com base no atributo atual e no valor do atributo na linha atual.
            gini = gini_index(groups, class_values)                                      # Calcula o índice Gini para a divisão.
            if gini < b_score:                                                           # Se o índice Gini calculado for menor que o melhor índice Gini encontrado até agora,
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups    # atualiza as variáveis para o melhor ponto de divisão encontrado até agora.

    return {'index': b_index, 'value': b_value, 'groups': b_groups}                      # Retorna um dicionário contendo o índice do atributo usado para a divisão, o valor do atributo usado para a divisão e os grupos resultantes da divisão.

In [13]:
# Criando um nó terminal em uma árvore de decisão:  --> quando a construção da árvore de decisão atinge um ponto onde não há mais divisões a serem feitas (por exemplo, todas as instâncias pertencem à mesma classe), um nó terminal é criado e atribuído a ele o valor mais comum da classe dentro desse grupo.

def to_terminal(group):                               # Define uma função chamada to_terminal (grupo de instâncias para o qual o nó terminal será criado).
    outcomes = [row[-1] for row in group]             # Obtém todas as classes das instâncias no grupo. O índice '-1' é usado para acessar a última coluna, que contém as classes.
    return max(set(outcomes), key=outcomes.count)     # Retorna a classe mais comum no grupo, que será atribuída ao nó terminal da árvore de decisão.

In [14]:
# Criando divisões nos nós de uma árvore de decisão: --> responsável por dividir um nó da árvore em dois nós filhos, criando a estrutura de decisão recursivamente.

def split(node, max_depth, min_size, n_features, depth): # Define uma função chamada split (nó atual que está sendo dividido, profundidade máxima permitida para a árvore, número mínimo de instâncias permitido em um nó antes de não ser mais dividido, número de características (atributos) a serem consideradas ao fazer uma divisão, profundidade atual na construção da árvore).

    left, right = node['groups']                                                # Obtém os grupos resultantes da divisão do nó atual,
    del(node['groups'])                                                         # e remove a chave 'groups' do dicionário node.

    # check for a no split (Verifica se não há dados à esquerda ou à direita para dividir. Nesse caso, o nó atual é transformado em um nó terminal com o valor da classe mais comum entre os dados combinados à esquerda e à direita).
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return

    # check for max depth (Verifica se a profundidade máxima da árvore foi atingida. Se sim, os nós à esquerda e à direita são transformados em nós terminais).
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    # process left child (1- Verifica se o número de instâncias no nó à esquerda é menor ou igual ao tamanho mínimo especificado).
    if len(left) <= min_size:                                             # Se sim, o nó à esquerda é transformado em um nó terminal.
        node['left'] = to_terminal(left)
    else:                                                                 # Caso contrário, a função get_split é chamada para encontrar a melhor divisão para o nó à esquerda, e o processo é repetido recursivamente.
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)

    # process right child (2- Verifica se o número de instâncias no nó à direita é menor ou igual ao tamanho mínimo especificado).
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [15]:
# Construindo uma árvore de decisão usando o algoritmo CART (Classification and Regression Trees):

def build_tree(train, max_depth, min_size, n_features):      # Define uma função chamada build_tree (conjunto de treinamento usado para construir a árvore de decisão, profundidade máxima permitida para a árvore, número mínimo de instâncias permitido em um nó antes de não ser mais dividido, número de características (atributos) a serem consideradas ao fazer uma divisão).
    root = get_split(dataset, n_features)                    # Usa a função get_split para encontrar a melhor divisão para o conjunto de treinamento, resultando no nó raiz da árvore de decisão.
    split(root, max_depth, min_size, n_features, 1)          # Usa a função split para dividir recursivamente o nó raiz e seus filhos até atingir os critérios de parada especificados pelos parâmetros max_depth e min_size.
    return root                                              # Retorna o nó raiz da árvore de decisão construída.

In [16]:
# Fazendo previsões usando uma árvore de decisão construída:

def predict(node, row):                                                                 # Define uma função chamada predict (nó atual da árvore de decisão, instância de dados (vetor) para a qual uma previsão será feita).

    if row[node['index']] < node['value']:                                              # Verifica se o valor do atributo na instância row é menor que o valor de divisão armazenado no nó. Se a condição for verdadeira, a travessia continua para o nó filho à esquerda.
        if isinstance(node['left'], dict): return predict(node['left'], row)            # 1- Se o nó filho for um dicionário, isso significa que ainda não é um nó terminal, então a função predict é chamada recursivamente para continuar a travessia pela árvore.
        else: return node['left']                                                       # 2- Se o nó filho for um valor terminal (ou seja, não é um dicionário), isso significa que é um nó terminal, e o valor do nó é retornado como a previsão.
    else:                                                                               # Caso contrário, a travessia continua para o nó filho à direita.
        if isinstance(node['right'], dict): return predict(node['right'], row)          # 1- ~~
        else: return node['right']                                                      # 2- ~~

In [17]:
# Criando uma amostra aleatória (subamostra) do conjunto de dados:

def subsample(dataset, ratio):                   # Define uma função chamada subsample (conjunto de dados original do qual a subamostra será criada, proporção de instâncias que serão incluídas na subamostra, representada como um valor entre 0 e 1).
    sample = list()                              # Cria uma lista vazia chamada sample, usada para armazenar as instâncias selecionadas aleatoriamente do conjunto de dados original.
    n_sample = round(len(dataset) * ratio)       # Calcula o tamanho da subamostra multiplicando o número total de instâncias no conjunto de dados pelo valor da proporção ratio. O resultado é arredondado para o inteiro mais próximo.

    while len(sample) < n_sample:                # Inicia um loop enquanto o tamanho da subamostra for menor que o tamanho desejado.
        index = randrange(len(dataset))          # Seleciona aleatoriamente um índice do conjunto de dados original usando a função randrange.
        sample.append(dataset[index])            # Adiciona a instância correspondente ao índice selecionado à subamostra.
    return sample                                # Retorna a subamostra criada.

In [18]:
# Fazendo previsões utilizando uma lista de árvores de decisão que foram treinadas em diferentes subamostras do conjunto de dados original (método conhecido como "bagging" (Bootstrap Aggregating)):

def bagging_predict(trees, row):                            # Define uma função chamada bagging_predict (lista de árvores de decisão previamente treinadas, instância de dados (vetor) para a qual uma previsão será feita).

    predictions = [predict(tree, row) for tree in trees]    # Usa uma compreensão de lista para iterar sobre cada árvore na lista trees e faz uma previsão para a instância row usando a função predict.
                                                            # Armazena todas as previsões em uma lista chamada predictions.

    return max(set(predictions), key=predictions.count)     # Usa a função set para obter os valores únicos das previsões.
                                                            # Usa a função max com o argumento key=predictions.count para encontrar a previsão mais comum (a moda) na lista de previsões.
                                                            # Retorna a previsão mais comum como a previsão final para a instância row.

In [19]:
# Algoritmo Random Forest: --> O algoritmo cria várias árvores de decisão em diferentes subamostras do conjunto de dados original e combina suas previsões para obter uma previsão final robusta e precisa.

def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()                                                                       # Cria uma lista vazia para armazenar as árvores de decisão que serão geradas.
    for i in range(n_trees):                                                             # Para cada árvore na floresta (total de n_trees árvores):
        sample = subsample(train, sample_size)                                           # Cria uma subamostra do conjunto de treinamento usando a função subsample com a proporção sample_size.
        tree = build_tree(sample, max_depth, min_size, n_features)                       # Constrói uma árvore de decisão usando a função build_tree com a subamostra criada e os parâmetros max_depth, min_size e n_features.
        trees.append(tree)                                                               # Adiciona a árvore construída à lista trees.
    predictions = [bagging_predict(trees, row) for row in test]                          # Faz previsões para cada instância no conjunto de teste usando a função bagging_predict, que combina as previsões de todas as árvores na floresta, e armazena as previsões em uma lista predictions.
    return(predictions)                                                                  # Retorna a lista de previsões para o conjunto de teste.

### Explicação Detalhada
- Subamostragem: Cada árvore é treinada em uma subamostra do conjunto de dados de treinamento, criada com reposição (ou seja, a mesma instância pode aparecer várias vezes na mesma subamostra).

- Seleção Aleatória de Características: Ao procurar a melhor divisão em cada nó de uma árvore, apenas um subconjunto aleatório de características é considerado (n_features). Isso promove a diversidade entre as árvores e ajuda a reduzir a correlação entre elas.

- Combinação das Previsões: As previsões de todas as árvores são combinadas usando votação majoritária (para classificação) ou média (para regressão) para produzir a previsão final.

Essa abordagem melhora a precisão e a robustez do modelo, tornando-o menos suscetível a overfitting em comparação com uma única árvore de decisão.

In [20]:
# Eliminando a coluna Accident_ID
dataset = dataset.drop['Accident_ID']

TypeError: 'method' object is not subscriptable

In [ ]:
print(dataset.head())  # Exibindo as primeiras linhas do DataFrame para checar


   Count Accident_Date Day_of_Week          Junction_Detail Accident_Injuries  \
0      0    2021-01-01    Thursday  T or staggered junction           Serious   
1      1    2021-01-05      Monday               Crossroads           Serious   
2      2    2021-01-04      Sunday  T or staggered junction            Slight   
3      3    2021-01-05      Monday  T or staggered junction           Serious   
4      4    2021-01-06     Tuesday               Crossroads           Serious   

        Light_Conditions  Number_of_Casualties  Number_of_Vehicles  \
0               Daylight                     1                   2   
1               Daylight                    11                   2   
2               Daylight                     1                   2   
3               Daylight                     1                   2   
4  Darkness - lights lit                     1                   2   

  Road_Surface_Conditions           Road_Type  Speed_limit Hour_of_Accident  \
0            

In [24]:
# Testando Algoritmo Random Forest:

seed(1)
# load and prepare data
dataset = load_csv("/content/car_accidentOFICIAL.csv")
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_flt(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('')

ValueError: could not convert string to float: 'Count'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Carregando os dados
dataset = pd.read_csv("/content/car_accidentOFICIAL.csv")

# Separando as features (atributos) e o target (classe)
X = dataset.drop(columns=['Accident_Injuries'])  # Features
y = dataset['Accident_Injuries']  # Variável de destino (classe)

# Dividindo os dados em conjuntos de treinamento e teste (por exemplo, 80% para treinamento e 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Definindo o modelo de Random Forest
random_forest_model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=1)

# Treinando o modelo
random_forest_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = random_forest_model.predict(X_test)

# Avaliando a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia:", accuracy)

ValueError: could not convert string to float: '201043R088080'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Carregando os dados
# Supondo que você já carregou seus dados em um DataFrame chamado 'dataset'

# Separando as features (atributos) e o target (classe)
X = dataset.drop(columns=['Accident_Injuries'])
y = dataset['Accident_Injuries']

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Definindo o modelo Random Forest
n_estimators = 10  # Número de árvores na floresta
max_depth = 10  # Profundidade máxima das árvores
random_forest_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=1)

# Treinando o modelo
random_forest_model.fit(X_train, y_train)

# Fazendo previsões
y_pred = random_forest_model.predict(X_test)

# Avaliando a precisão do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


ValueError: could not convert string to float: '201043R088080'